In [9]:
"""
Juliann Booth
Homework 1
June 22, 2016
"""

from IPython.display import display, Latex, Math, Markdown
import numpy as np
import sympy as sym
sym.init_printing()
def printltx(s):
   try: 
       display(Latex(s))
   except: 
       print(s)
def ltxmtx(A, axis = 1):
       try: 
           if isinstance(A, np.ndarray):
               A = A.round(4)
           M = sym.Matrix(A)
           sh = np.array(M.shape)
           sz = np.prod(sh)
           if sz == np.max(sh):
               if axis == 0:
                   M=M.reshape(sz,1)
               else:
                   M=M.reshape(1,sz)
           return " $"+ sym.latex(M) + "$ "
       except: 
           return A
       
array_f = get_ipython().display_formatter.formatters['text/latex']
array_f.for_type('numpy.ndarray',ltxmtx)
array_f.for_type('sympy.matrices.dense.MutableDenseMatrix', ltxmtx)


"""method 1: simulate a large number of samples"""
def simulate_samples(P, phi, Steps, Records, Samples):
    """Evolves the system by simulating many sample paths"""
    print("\n")
    printltx(r"Method 1: Simulating "+str(Steps)+" steps with " +str(Samples)+" samples.")
    epdf = np.zeros([Records,States],dtype=float)
    epdf[0,:] = phi[:]
    
    y = np.rint(phi*Samples).astype(int)
    Samples = np.sum(y)
    X = np.zeros([Records,Samples], dtype=int)
    
    i1=0
    for j in range(States):
        i2 = i1 + y[j]
        X[0,i1:i2]=j
        i1=i2
    np.random.shuffle(X[0,:])
    
    for step in range(Steps):
        current_step = step%Records
        next_step = (step+1)%Records
        for samp in range(Samples):
            current_state = X[current_step, samp]
            r=np.random.rand()
            for next_state in range(States):
                r-= P[current_state,next_state]
                if r <0 :
                    X[next_step,samp] = next_state
                    break
        epdf[next_step,:] = np.histogram(X[next_step,:], normed=True, bins=range(States+1))[0]
    epdf = np.roll(epdf,Records-next_step-1,axis=0)
    printltx(r"I did "+str(Steps)+" steps with "+str(Samples)+" samples. The last "+str(Records)+" distributions are written as rows below.")
    #display(epdf)
    printltx(ltxmtx(epdf))
    printltx(r"The average of these distributions is "+ltxmtx(np.mean(epdf,0)))
        
"""Method 2: evolve the distribution using matrix multiplication"""
def matrix_evolution(P, phi, Steps, Records):
    """Evolves the system via matrix multiplication"""
    print("\n")
    printltx(r"Method 2: Evolving "+str(Steps)+" steps via matrix multiplication.")
    tpdf = np.zeros([Records,States],dtype=float)
    tpdf[0,:]=phi[:]
    for step in range(Steps):
        current_step=step%Records
        next_step = (step+1)%Records
        tpdf[next_step,:] = tpdf[current_step,:].dot(P)
    tpdf=np.roll(tpdf, Records-next_step-1, axis=0)
    printltx(r"I did "+str(Steps)+" steps. The last "+str(Records)+ " distributions are written as rows below.")
    #display(tpdf)
    printltx(ltxmtx(tpdf))
    printltx(r"The average of these distributions is "+ltxmtx(np.mean(tpdf,0)))
    
"""Function to make sure that eigenvectors are normalized since Python will usually give scalar multiples"""
def normalize_rows(L):
    tol=0.001
    n=L.shape[0]
    for i in range(n):
        s=np.sum(L[i,:])
        if abs(s) > tol:
            L[i,:]*=1/s
        else:
            for j in range(n):
                if abs(L[i,j]) > tol:
                    L[i,:] *= 1/L[i,j]
                    break
    return L

"""Method 3: Use eigenvectors/eigenvalues"""
def eigen_future(P,phi):
    """Long term behavior via eigenvectors and eigenvalues"""
    print("\n")
    printltx(r"Method 3: Using eigenvectors and eigenvalues to find long term behavior.")
    (evals,RT) = np.linalg.eig(P.T)
    L=RT.T
    """sort eigenvalues so that largest is first and descends with length"""
    Len = np.abs(evals).round(4)
    Re = np.real(evals).round(4)
    Im = np.imag(evals).round(4)
    sort = np.lexsort([Im,Re,Len]) 
    sort = sort[::-1] #this is needed because lexsort sorts in ascending but we want descending.
    evals = evals[sort]
    L = L[sort]
    Len=Len[sort]
            
    L = normalize_rows(L)
    R=np.linalg.inv(L)
    D=np.diag(evals)
    printltx(r"left eigenvectors = rows of "+ltxmtx(L)+", $\lambda =$"+ltxmtx(evals,0)+", $|\lambda| =$"+ltxmtx(np.abs(evals),0)) 
    
    States = P.shape[0]
    trans = Len<1
    Period = States-np.sum(trans)
    Dp=D.copy()
    Dp[:,trans] = 0
    pdf = np.zeros([Period, States])
    for step in range(Period):
        pdf[step,:] = phi.dot(R).dot(Dp**(step+1)).dot(L)
    printltx(r"The analysis via eigenvectors says that, in the long run, this system should land in the distribution below (If more than one row, the system will land in only one).")
    #display(pdf)
    printltx(ltxmtx(pdf))
    printltx(r"The average of these distributions is "+ltxmtx(np.mean(pdf,0)))
    return R,D,L

In [10]:
"""Create a 5x5 matrix that is a biased, non-lazy random walk for boundary condition absorbing"""
States =5
p = 0.6
q = 1-p
phi = np.array([0,0,1,0,0]) #initial distribution
P_absorb = np.array([[1,0,0,0,0],[q,0,p,0,0],[0,q,0,p,0],[0,0,q,0,p],[0,0,0,0,1]])


Steps = 1000
Records = 15
Samples = 1000

printltx("Below are examples using an Absorbing Boundary Condition")
simulate_samples(P_absorb, phi, Steps, Records, Samples)
matrix_evolution(P_absorb, phi, Steps, Records)
eigen_future(P_absorb, phi)



#Now make the boundary condition be reflecting
P_reflect = np.array([[0,1,0,0,0],[q,0,p,0,0],[0,q,0,p,0],[0,0,q,0,p],[0,0,0,1,0]])

printltx("Below are examples using a Reflecting Boundary Condition")
simulate_samples(P_reflect, phi, Steps, Records, Samples)
matrix_evolution(P_reflect, phi, Steps, Records)
eigen_future(P_reflect, phi)




#Now make the boundary semi-reflecting:
P_semi = np.array([[q,p,0,0,0],[q,0,p,0,0],[0,q,0,p,0],[0,0,q,0,p],[0,0,0,q,p]])
printltx("Below are examples using a Semi-Reflecting Boundary Condition")
simulate_samples(P_semi, phi, Steps, Records, Samples)
matrix_evolution(P_semi, phi, Steps, Records)
eigen_future(P_semi, phi)

print
             
            


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>